In [26]:
import gzip
import json
import tensorflow as tf
import collections

tf.logging.set_verbosity(tf.logging.INFO)

In [38]:
DATA_DIR = "data/v1.0/dev/nq-dev-0*.jsonl.gz"
#DATA_DIR = "data/tiny-dev/nq-dev-sample.jsonl.gz"
max_position = 50
skip_nested_contexts = False
TextSpan = collections.namedtuple("TextSpan", "token_positions text")

In [39]:

examples = []
with gzip.GzipFile(DATA_DIR) as f:
    for line in f:
        example = json.loads(line,'utf-8')
        #print example['example_id']
        a = example['annotations']
        #long_answers = [ a['long_answer'] for a in example['annotations'] if a['long_answer']['start_byte'] >= 0]
        print a
        '''
        long_answer_bounds = [
          (la['start_byte'], la['end_byte']) for la in long_answers
          ]
        #print long_answer_bounds
        long_answer_counts = [
          long_answer_bounds.count(la) for la in long_answer_bounds
          ]
        '''
        print(example["long_answer_candidates"])
        #print long_answer_counts
        #print example.keys()
        #print "short_answers", a["short_answers"]
        #if len(a["short_answers"]) > 1 and len(a["long_answer"]) > 1:
        #    print (a)
        ''' if len(a["short_answers"]) > 1:
            print long_answers
            print long_answer_counts
            print "short_answers", a["short_answers"]
        '''
        examples.append(example)
        break
        

IOError: [Errno 2] No such file or directory: 'data/v1.0/dev/nq-dev-0*.jsonl.gz'

In [40]:
CANDIDATES_COUNT = {"Paragraph": 0, "Table" : 0, "List" : 0, "Other" : 0}
def get_text_span(example, span):
    """Returns the text in the example's document in the given token span."""
    token_positions = []
    tokens = []
    for i in range(span["start_token"], span["end_token"]):
        t = example["document_tokens"][i]
        if not t["html_token"]:
            token_positions.append(i)
            token = t["token"].replace(" ", "")
            tokens.append(token)
    return TextSpan(token_positions, " ".join(tokens))

def get_candidate_text(e, idx):
    """Returns a text representation of the candidate at the given index."""
    # No candidate at this index.
    if idx < 0 or idx >= len(e["long_answer_candidates"]):
        return TextSpan([], "")
    # This returns an actual candidate.
    return get_text_span(e, e["long_answer_candidates"][idx])

def should_skip_context(e, idx):
    t = get_candidate_type(e, idx)
    CANDIDATES_COUNT[t] += 1
    if (skip_nested_contexts and
        not e["long_answer_candidates"][idx]["top_level"]):
        return True
    elif not get_candidate_text(e, idx).text.strip():
        # Skip empty contexts.
        return True
    else:
        return False

def candidates_iter(e):
    """Yield's the candidates that should not be skipped in an example."""
    for idx, c in enumerate(e["long_answer_candidates"]):
        if should_skip_context(e, idx):
            continue
    yield idx, c

def get_candidate_type(e, idx):
    """Returns the candidate's type: Table, Paragraph, List or Other."""
    c = e["long_answer_candidates"][idx]
    first_token = e["document_tokens"][c["start_token"]]["token"]
    if first_token == "<Table>":
        return "Table"
    elif first_token == "<P>":
        return "Paragraph"
    elif first_token in ("<Ul>", "<Dl>", "<Ol>"):
        return "List"
    elif first_token in ("<Tr>", "<Li>", "<Dd>", "<Dt>"):
        return "Other"
    else:
        tf.logging.warning("Unknoww candidate type found: %s", first_token)
    return "Other"

def add_candidate_types_and_positions(e):
    """Adds type and position info to each candidate in the document."""
    counts = collections.defaultdict(int)
    for idx, c in candidates_iter(e):
        context_type = get_candidate_type(e, idx)
        if counts[context_type] < max_position:
            counts[context_type] += 1
            c["type_and_position"] = "[%s=%d]" % (context_type, counts[context_type])

def load_example(line):
    e = json.loads(line, object_pairs_hook=collections.OrderedDict)
    add_candidate_types_and_positions(e)
    
def read_nq_examples(input_file, is_training):
    """Read a NQ json file into a list of NqExample."""
    input_paths = tf.gfile.Glob(input_file)
    input_data = []

    def _open(path):
        if path.endswith(".gz"):
            return gzip.GzipFile(fileobj=tf.gfile.Open(path, "r"))
        else:
            return tf.gfile.Open(path, "r")

    for path in input_paths:
        tf.logging.info("Reading: %s", path)
        with _open(path) as input_file:
            for line in input_file:
                load_example(line)

In [ ]:
read_nq_examples(DATA_DIR, False)
tf.logging.info("Disrtribution of long answer types: %s",CANDIDATES_COUNT)

I0916 15:49:41.950619 140638234494784 <ipython-input-40-91a87fe9d373>:82] Reading: data/v1.0/dev/nq-dev-04.jsonl.gz
